In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd # 2.x to support pyarrow
import pyarrow as pa
import openpyxl # for reading xlsx with structure
import numpy as np
import plotly.express as px
import math
import re

In [ ]:
messages = pd.read_excel('../data/source/messages.xlsx', index_col=0)

In [ ]:
# messages

In [ ]:
x_train = pd.read_parquet('../data/source/X_train.parquet', engine='pyarrow', dtype_backend='pyarrow')
x_train['DATASET_IDX'] = range(0, len(x_train))

In [ ]:
# x_train

In [ ]:
x_test = pd.read_parquet('../data/source/X_test.parquet', engine='pyarrow', dtype_backend='pyarrow')
x_test['DATASET_IDX'] = range(0, len(x_test))

In [ ]:
# x_test

In [ ]:
y_train = pd.read_parquet('../data/source/y_train.parquet', engine='pyarrow', dtype_backend='pyarrow')
y_train['DATASET_IDX'] = range(0, len(y_train))

In [ ]:
# y_train

In [ ]:
def clean_x_machine_name(dirty):
    value = dirty
    value = re.sub(r'ЭКСГАУСТЕР (\d+)', r'ЭКСГАУСТЕР А/М №\1', value)
    value = re.sub(r'\s+', ' ', value)
    value = value.strip()
    return value

def clean_x_meter_name(dirty):
    value = dirty
    value = re.sub(r'ТОК РОТОРА(\d+)', r'ТОК РОТОРА \1', value)
    value = re.sub(r'\s+', ' ', value)
    value = value.strip()
    return value

def extract_x_structure(x):
    x_columns = pd.DataFrame({ 'ИМЯ МАШИНЫ И НАЗВАНИЕ ДАТЧИКА': list(set(x.columns) - set(['DATASET_IDX'])) })
    x_columns['ИМЯ МАШИНЫ'] = x_columns['ИМЯ МАШИНЫ И НАЗВАНИЕ ДАТЧИКА'].str.extract(r'(ЭКСГАУСТЕР \d+)')
    x_columns['ИМЯ МАШИНЫ'] = x_columns['ИМЯ МАШИНЫ'].map(clean_x_machine_name)
    x_columns['НАЗВАНИЕ ДАТЧИКА'] = x_columns['ИМЯ МАШИНЫ И НАЗВАНИЕ ДАТЧИКА'].str.extract(r'ЭКСГАУСТЕР \d+\. (.*)$')
    x_columns['НАЗВАНИЕ ДАТЧИКА'] = x_columns['НАЗВАНИЕ ДАТЧИКА'].map(clean_x_meter_name)
    return x_columns.pivot(index='НАЗВАНИЕ ДАТЧИКА', columns='ИМЯ МАШИНЫ', values='ИМЯ МАШИНЫ И НАЗВАНИЕ ДАТЧИКА')

# x_structure = extract_x_structure(x_train)
# x_structure.to_parquet('../data/pipeline/x_structure.parquet')
x_structure = pd.read_parquet('../data/pipeline/x_structure.parquet')

In [ ]:
# x_structure

In [ ]:
def clean_y_machine_name(dirty):
    value = dirty
    value = re.sub(r'Y_ЭКСГАУСТЕР А/М №(\d+)', r'ЭКСГАУСТЕР А/М №\1', value)
    value = re.sub(r'\s+', ' ', value)
    value = value.strip()
    return value

def clean_y_place_name(dirty):
    value = dirty
    value = re.sub(r'ЭКСГАУСТЕРА №\d+', '', value)
    value = re.sub(r'ЭКСГАУСТЕРА А/М №\d+', '', value)
    value = re.sub(r'ЭКСГ. №\d+', '', value)
    value = re.sub(r'ЭКСГ.№\d+', '', value)
    value = re.sub(r'ЭКСГ\d+', '', value)
    value = re.sub(r'ТР№\d+', 'ТР', value)
    value = re.sub(r'^ЭКСГАУСТЕР А/М №\d+', 'ЭКСГАУСТЕР А/М', value)
    value = re.sub(r'^ЭКСГАУСТЕР Н-8000 А/М №\d+', 'ЭКСГАУСТЕР Н-8000 А/М', value)
    value = re.sub(r'\s+', ' ', value)
    value = value.strip()
    return value

def extract_y_structure(y):
    y_columns = pd.DataFrame({ 'ИМЯ МАШИНЫ И НАЗВАНИЕ ТЕХ МЕСТА': list(set(y.columns) - set(['DATASET_IDX'])) })
    y_columns['ИМЯ МАШИНЫ'] = y_columns['ИМЯ МАШИНЫ И НАЗВАНИЕ ТЕХ МЕСТА'].str.extract(r'(Y_ЭКСГАУСТЕР А/М №\d+)')
    y_columns['ИМЯ МАШИНЫ'] = y_columns['ИМЯ МАШИНЫ'].map(clean_y_machine_name)
    y_columns['НАЗВАНИЕ ТЕХ МЕСТА'] = y_columns['ИМЯ МАШИНЫ И НАЗВАНИЕ ТЕХ МЕСТА'].str.extract(r'Y_ЭКСГАУСТЕР А/М №\d+_(.*)$')
    y_columns['НАЗВАНИЕ ТЕХ МЕСТА'] = y_columns['НАЗВАНИЕ ТЕХ МЕСТА'].map(clean_y_place_name)
    return y_columns.pivot(index='НАЗВАНИЕ ТЕХ МЕСТА', columns='ИМЯ МАШИНЫ', values='ИМЯ МАШИНЫ И НАЗВАНИЕ ТЕХ МЕСТА')

# y_structure = extract_y_structure(y_train)
# y_structure.to_parquet('../data/pipeline/y_structure.parquet')
y_structure = pd.read_parquet('../data/pipeline/y_structure.parquet')

In [ ]:
# y_structure

In [ ]:
messages_pretty = pd.DataFrame(index=messages.index)

messages_pretty['ИМЯ МАШИНЫ'] = messages['ИМЯ_МАШИНЫ']
messages_pretty['НАЗВАНИЕ ТЕХ МЕСТА'] = messages['НАЗВАНИЕ_ТЕХ_МЕСТА'].map(clean_y_place_name)
messages_pretty['НАЗВАНИЕ ТЕХ МЕСТА ПОЛНОЕ'] = messages['НАЗВАНИЕ_ТЕХ_МЕСТА']
messages_pretty['ВИД СООБЩЕНИЯ'] = messages['ВИД_СООБЩЕНИЯ']
messages_pretty['ОПИСАНИЕ'] = messages['ОПИСАНИЕ']
messages_pretty['ДАТА НАЧАЛА НЕИСПРАВНОСТИ'] = messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ']
messages_pretty['ДАТА УСТРАНЕНИЯ НЕИСПРАВНОСТИ'] = messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ']
messages_pretty['ДЛИТЕЛЬНОСТЬ НЕИСПРАВНОСТИ, СЕК'] = (messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'] - messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ']).dt.seconds
messages_pretty['ТЕКСТ ГРУППЫ КОДОВ'] = messages['ТЕКСТ_ГРУППЫ_КОДОВ']

# @NOTE: Согласно комментарию эксперта, незакрытые неисправности в тренировочном наборе это скорее ошибка заполнения
messages_pretty = messages_pretty[messages_pretty['ДАТА УСТРАНЕНИЯ НЕИСПРАВНОСТИ'].notna()]

# @NOTE: Согласно комментарию эксперта, запланированные остановки не нужно предсказывать
messages_pretty = messages_pretty[messages_pretty['ТЕКСТ ГРУППЫ КОДОВ'] != 'ТЕХНИЧЕСКИЕ ПЛАНОВЫЕ']

In [ ]:
# messages_pretty

In [ ]:
def prettify_x(x):
    x_pretty = pd.DataFrame()
    for (machine, meter_names) in x_structure.transpose().iterrows():
        x_pretty_slice = pd.DataFrame(index=x.index)
        x_pretty_slice['ROW_IDX'] = 0
        x_pretty_slice['DATASET_IDX'] = x['DATASET_IDX']
        x_pretty_slice['ИМЯ МАШИНЫ'] = machine
        x_pretty_slice[meter_names.index] = x[meter_names.values]
        x_pretty_slice = x_pretty_slice.reset_index()
        x_pretty = pd.concat([x_pretty, x_pretty_slice])
    x_pretty['ROW_IDX'] = range(0, len(x_pretty))
    return x_pretty

In [ ]:
# x_train_pretty = prettify_x(x_train)
# x_train_pretty.to_parquet('../data/pipeline/x_train_pretty.parquet')
x_train_pretty = pd.read_parquet('../data/pipeline/x_train_pretty.parquet')

# x_train_pretty_1d = x_train_pretty.set_index(['ИМЯ МАШИНЫ', 'DT']) \
#     .groupby([pd.Grouper(level='ИМЯ МАШИНЫ'), pd.Grouper(level='DT', freq='1d')]).mean().reset_index()
# x_train_pretty_1d.to_parquet('../data/pipeline/x_train_pretty_1d.parquet')
x_train_pretty_1d = pd.read_parquet('../data/pipeline/x_train_pretty_1d.parquet')

# x_train_pretty_1h = x_train_pretty.set_index(['ИМЯ МАШИНЫ', 'DT']) \
#     .groupby([pd.Grouper(level='ИМЯ МАШИНЫ'), pd.Grouper(level='DT', freq='1h')]).mean().reset_index()
# x_train_pretty_1h.to_parquet('../data/pipeline/x_train_pretty_1h.parquet')
x_train_pretty_1h = pd.read_parquet('../data/pipeline/x_train_pretty_1h.parquet')

In [ ]:
# x_train_pretty

In [ ]:
# px.line(x_train_pretty_1h, x='DT', y=x_structure.index, facet_row='ИМЯ МАШИНЫ', height=2000)
# px.line(x_train_pretty_1d, x='DT', y=x_structure.index, facet_row='ИМЯ МАШИНЫ', height=2000)

In [ ]:
# x_test_pretty = prettify_x(x_test)
# x_test_pretty.to_parquet('../data/pipeline/x_test_pretty.parquet')
x_test_pretty = pd.read_parquet('../data/pipeline/x_test_pretty.parquet')

# x_test_pretty_1d = x_test_pretty.set_index(['ИМЯ МАШИНЫ', 'DT']) \
#     .groupby([pd.Grouper(level='ИМЯ МАШИНЫ'), pd.Grouper(level='DT', freq='1d')]).mean().reset_index()
# x_test_pretty_1d.to_parquet('../data/pipeline/x_test_pretty_1d.parquet')
x_test_pretty_1d = pd.read_parquet('../data/pipeline/x_test_pretty_1d.parquet')

# x_test_pretty_1h = x_test_pretty.set_index(['ИМЯ МАШИНЫ', 'DT']) \
#     .groupby([pd.Grouper(level='ИМЯ МАШИНЫ'), pd.Grouper(level='DT', freq='1h')]).mean().reset_index()
# x_test_pretty_1h.to_parquet('../data/pipeline/x_test_pretty_1h.parquet')
x_test_pretty_1h = pd.read_parquet('../data/pipeline/x_test_pretty_1h.parquet')

In [ ]:
# x_test_pretty

In [ ]:
# px.line(x_test_pretty_1h, x='DT', y=x_structure.index, facet_row='ИМЯ МАШИНЫ', height=2000)
# px.line(x_test_pretty_1d, x='DT', y=x_structure.index, facet_row='ИМЯ МАШИНЫ', height=2000)

In [ ]:
def prettify_y(y):
    y_pretty = pd.DataFrame()
    for (machine, meter_names) in y_structure.transpose().iterrows():
        present_meter_names = meter_names[meter_names.notna()]
        y_pretty_slice = pd.DataFrame(index=y.index)
        y_pretty_slice['ROW_IDX'] = 0
        y_pretty_slice['DATASET_IDX'] = y['DATASET_IDX']
        y_pretty_slice['ИМЯ МАШИНЫ'] = machine
        y_pretty_slice[present_meter_names.index] = y[present_meter_names.values]
        y_pretty_slice = y_pretty_slice.reset_index()
        y_pretty = pd.concat([y_pretty, y_pretty_slice])
    y_pretty['ROW_IDX'] = range(0, len(y_pretty))
    return y_pretty

In [ ]:
# y_train_pretty = prettify_y(y_train)
# y_train_pretty.to_parquet('../data/pipeline/y_train_pretty.parquet')
y_train_pretty = pd.read_parquet('../data/pipeline/y_train_pretty.parquet')

In [ ]:
# y_train_pretty